<h1 align='center' style='color:purple'><b>Sentiment Analysis</b></h1>

Sentiment analysis is the area which deals with judgments, responses as well as feelings, which is generated from texts, being extensively used in fields like data mining, web mining, and social media analytics because sentiments are the most essential characteristics to judge the human behavior. Sentiments can be positive, negative, or neutral.

<b> Install tweepy </b>  
[Tweepy](https://docs.tweepy.org/en/latest/) is an easy-to-use Python library for accessing the Twitter API.

In [1]:
!pip install tweepy

  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)


<b>Import tweepy and pandas Library </b>

In [1]:
import tweepy
import pandas as pd

Go to https://developer.twitter.com/en and get your own consumer key, consumer key secret, access token, access token secret. It is free of cost (Limited use).

In [2]:
# Save the keys and token obtained from twitter

consumer_key = '2dq4wMRXvtNn8nFRx5V5qR6YX'
consumer_secret = 'pLm7uEIyw6LEZdi7z51dFNOK7hAJTnSyDLqSFdiCdugcPcCOt2'
access_token = '1382592322903314432-pVmJCnt2eYRSGRQqI0pzlxUfrSgzaR'
access_token_secret = 'mnaYpOqOVHR1r45C07o3SjytVtASXb25vBcjLXeul7P7R'

# api_key="2dq4wMRXvtNn8nFRx5V5qR6YX"
# api_secret_key="pLm7uEIyw6LEZdi7z51dFNOK7hAJTnSyDLqSFdiCdugcPcCOt2"
# bearer_token="AAAAAAAAAAAAAAAAAAAAAEN6OgEAAAAA8tgS1XdhAytyd71YR%2BRLdZW0nnU%3DjzEsCj0XQuccvpkT0cN0YeDqWa9w1GiPrSnFeBdBlEEq1UR5yJ"

<b> Authentication using the obtained keys and tokens</b>

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

<b> Importing 150 tweets from BBC News twitter handle (@BBCBreaking)</b>

In [30]:
n_tweets = 150
get_tweets = tweepy.Cursor(api.user_timeline, id = 'BBCBreaking', tweet_mode = 'extended').items(n_tweets)

150 tweet details relevant to the keyword will be stored in get_tweets variable in JSON format

<b> Iterate over extracted tweets saved in 'get_tweets' variable and store the tweet's text and time</b>

In [31]:
tweets = []
time_of_tweet = []
for i in get_tweets:
    tweets.append(i.full_text)            # .full_text returns the text written in a particular tweet
    time_of_tweet.append(i.created_at)    # .created_at returns the time of tweet

You can read about more tweepy attributes [here](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet)

<b> Store the Data in a DataFrame </b>

In [32]:
df = pd.DataFrame({'tweet':tweets, 'time':time_of_tweet})

<b> First 10 tweets </b>

In [33]:
df.iloc[:10,:]

,tweet,time
0,The Duke of Edinburgh's four children will wal...,2021-04-15 16:02:04
1,Former police officer Derek Chauvin accused of...,2021-04-15 15:03:31
2,Officer who shot dead motorist Daunte Wright n...,2021-04-14 16:31:43
3,Denmark becomes the first country to permanent...,2021-04-14 12:46:13
4,US President Joe Biden set to announce troops ...,2021-04-13 16:28:07
5,All those in UK’s top nine Covid priority grou...,2021-04-12 21:35:51
6,"""During this encounter... the officer drew the...",2021-04-12 18:21:08
7,"Killing of Daunte Wright, a black man shot in ...",2021-04-12 18:12:26
8,Prince Harry says his grandfather was a man of...,2021-04-12 13:45:51
9,Prince William pays tribute to the “extraordin...,2021-04-12 13:12:33


<b> Remove Re-tweets </b>

In [34]:
# Remove Re-tweets
df = df[df['tweet'].str.contains("RT")==False]

<b> Reset Index </b>

In [35]:
# Reset index as there are some irregularities in index after removal of some rows
df = df.reset_index().drop('index' ,axis = 1)

<b> Remove Links as they dont represent sentiment </b>

In [36]:
# Use Regex Library to find the pattern "http" followed any length of sting or number with white space and replace with "".
import re

# Keep df as backup, number of request you make from API is limited.
df1 = df.copy()
df1.tweet = df.tweet.apply(lambda x : re.sub(r"http\S+", "", x))

<b> Remove '/n' wish represents line breaks </b>

In [37]:
df1.tweet = df1.tweet.apply(lambda x : x.replace('\n\n',' '))
df1.tweet = df1.tweet.apply(lambda x : x.replace('\n \n',' '))

<b> TextBlob </b>  
[TextBlob](https://textblob.readthedocs.io/en/dev/) is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

<b> Install Textblob </b>

In [25]:
!pip install textblob

<b> Import Textblob and get tweet sentiment score and save it in 'sentimen' column of df1 </b>

In [29]:
from textblob import TextBlob
df1['sentiment'] = df1['tweet'].apply(lambda x: TextBlob(x).sentiment)
df1

,tweet,time,sentiment
0,Former police officer Derek Chauvin accused of...,2021-04-15 15:03:31,"(0.0, 0.0)"
1,Officer who shot dead motorist Daunte Wright n...,2021-04-14 16:31:43,"(-0.05, 0.4)"
2,Denmark becomes the first country to permanent...,2021-04-14 12:46:13,"(0.275, 0.6166666666666667)"
3,US President Joe Biden set to announce troops ...,2021-04-13 16:28:07,"(0.0, 0.0)"
4,All those in UK’s top nine Covid priority grou...,2021-04-12 21:35:51,"(0.375, 0.41666666666666663)"
...,...,...,...
137,“It’s hard to compute the sorrow contained in ...,2021-01-26 17:18:26,"(-0.6458333333333334, 0.7708333333333333)"
138,As the number of people who have died with Cov...,2021-01-26 16:50:02,"(-0.5, 0.875)"
139,"More than 100,000 people have died with corona...",2021-01-26 16:41:04,"(0.24242424242424243, 0.34848484848484845)"
140,Janet Yellen confirmed as first female US Trea...,2021-01-25 23:46:01,"(0.21666666666666667, 0.5)"


The .sentiment attribute returns 2 things :  
 - sentiment [-1,1] : 0 means neutral, <0 means negative and >0 means positive sentiment
 - Subjectiveness : tell if the topic being mentioned in the tweet is an opinion or fact.
 
Since we have to check tweet's sentiment, we will use the sentiment score and classify it as positive, negative or neutral using a nested if-else statement.

In [28]:
df1.sentiment = df1.sentiment.apply(lambda x : "Positive" if x[0] > 0 else ("Negative" if x[0] < 0 else "Neutral"))
df1

,tweet,time,sentiment
0,Former police officer Derek Chauvin accused of...,2021-04-15 15:03:31,Neutral
1,Officer who shot dead motorist Daunte Wright n...,2021-04-14 16:31:43,Negative
2,Denmark becomes the first country to permanent...,2021-04-14 12:46:13,Positive
3,US President Joe Biden set to announce troops ...,2021-04-13 16:28:07,Neutral
4,All those in UK’s top nine Covid priority grou...,2021-04-12 21:35:51,Positive
...,...,...,...
137,“It’s hard to compute the sorrow contained in ...,2021-01-26 17:18:26,Negative
138,As the number of people who have died with Cov...,2021-01-26 16:50:02,Negative
139,"More than 100,000 people have died with corona...",2021-01-26 16:41:04,Positive
140,Janet Yellen confirmed as first female US Trea...,2021-01-25 23:46:01,Positive


The sentiments corresponding to the tweets are stored in the sentiment column.